In [1]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
import requests
import re

In [2]:
driver=webdriver.Chrome(r"C:\Users\nisha\Desktop\data science\chromedriver_win32\chromedriver.exe")

In [3]:
driver.get('https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1')

In [4]:
# Creating emply lists for extraction
sno=[]
movie_name=[]
director_name_1=[]
director_name_2=[]
director_name_3=[]
duration=[]
year=[]
ratings=[]
metascore=[]

stars_1=[]
stars_2=[]
stars_3=[]
stars_4=[]
votes=[]
genre_1=[]
genre_2=[]
genre_3=[]
gross_collection=[]
certification=[]
url=[]

In [5]:
try:
    for page in range(0,50):
        
        #To extract S. no. , Movie name and year
        s=driver.find_elements(By.XPATH,'//h3[@class="lister-item-header"]')
        for i in s:
            sno.append(i.text.split(".")[0])
            movie_name.append(i.text.split(".")[1].split("(")[0].strip(" "))
            year.append(i.text.split('(')[-1].strip(")"))
            
        # To extract Duration
        dur=driver.find_elements(By.XPATH,'//span[@class="runtime"]')
        for i in dur:
            duration.append(i.text.split(" ")[0])
        
        # To extract metascore
        meta=driver.find_elements(By.XPATH,'//div[@class="ratings-bar"]')
        for i in meta:
            try:
                metascore.append(i.text.split("\n")[1].split(" ")[0])
            except:
                metascore.append("Nan")

        #To extract ratings
        rat=driver.find_elements(By.TAG_NAME,'strong')
        for i in rat[2:]:
            ratings.append(i.text)
            
        # To extract director names with sub-fields.
        dire=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]')
        for i in dire:
            director_text = i.text
            directors = re.findall(r"Director[s]?:\s+(.*)", director_text)
            if directors:             
                director_name_1.append(directors[-1].split("|")[0].split(",")[0])
                try:
                    director_name_2.append(directors[-1].split("|")[0].split(",")[1])
                except:
                    director_name_2.append("nan")
                try:
                    director_name_3.append(directors[-1].split("|")[0].split(",")[2])
                except:
                    director_name_3.append("nan")


        # TO extract votes and gross collection
        v=driver.find_elements(By.XPATH,'//p[@class="sort-num_votes-visible"]')
        for i in v:
            votes.append(i.text.split(":")[1].split(" ")[1])
        for i in v:
            try:
                gross_collection.append(i.text.split("|")[1].split("$")[1].strip("M"))
            except:
                gross_collection.append("Nan")

        # To extract certification
        cer=driver.find_elements(By.XPATH,'//p[@class="text-muted "]')
        for i in cer:
            try:
                (i.text.split("|")[2]) # To verify extraction condition
                certification.append(i.text.split("|")[0])
            except:
                certification.append("Nan")

        # To extract genres with sub-fields
        gen=driver.find_elements(By.XPATH,'//span[@class="genre"]')
        for i in gen:
            genre_1.append(i.text.split(",")[0])
            try:
                genre_2.append(i.text.split(",")[1])
            except:
                genre_2.append("Nan")
            try:
                genre_3.append(i.text.split(",")[2])
            except:
                genre_3.append("Nan")


        # To extract stars names with sub-fields
        star=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]')
        for i in dire:
            director_text = i.text
            directors = re.findall(r"Director[s]?:\s+(.*)", director_text)
            if directors:    
                stars_1.append(directors[-1].split(":")[1].split(",")[0])
                try:
                    stars_2.append(directors[-1].split(":")[1].split(",")[1])
                except:
                    stars_2.append("Nan")
                try:
                    stars_3.append(directors[-1].split(":")[1].split(",")[2])
                except:
                    stars_3.append("Nan")
                try:
                    stars_4.append(directors[-1].split(":")[1].split(",")[3])
                except:
                    stars_4.append("Nan")

        # To extract URL of all movies to extract popularity data which is given inside of each movie page.
        url_t =driver.find_elements(By.XPATH, '//h3[@class="lister-item-header"]//a')
        for i in url_t:
            url.append(i.get_attribute("href"))


        search_button=driver.find_element(By.XPATH,'//a[@class="lister-page-next next-page"]')
        search_button.click()
        time.sleep(1)
except:
    print("Data Extracted")

Data Extracted


In [10]:
dataset1=pd.DataFrame({"S No.":sno, "Movie Name":movie_name,"Duration":duration,"Year":year,"Rating":ratings,"Metascore":metascore, "Director1":director_name_1,"Director2":director_name_2,"Director3":director_name_3})
dataset1

,S No.,Movie Name,Duration,Year,Rating,Metascore,Director1,Director2,Director3
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.1,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson
1,2,The Dark Knight,152,2008,9.0,84,Christopher Nolan,nan,nan
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,nan,nan
3,4,Inception,148,2010,8.8,74,Christopher Nolan,nan,nan
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,nan,nan
...,...,...,...,...,...,...,...,...,...
1746,"1,747",Radhe,109,2021,1.9,Nan,Prabhu Deva,nan,nan
1747,"1,748",Race 3,160,2018,1.9,Nan,Remo D'Souza,nan,nan
1748,"1,749",Angels Apocalypse,85,2015,1.7,Nan,Sean Cain,Enzo Zelocchi,nan
1749,"1,750",The Cost of Deception,125,2021,1.5,Nan,Keith English,nan,nan


In [11]:
Dataset2=pd.DataFrame({"Movie Name":movie_name, "Star1":stars_1,"Star2":stars_2,"Star3":stars_3,"Star4":stars_4,"Genre1":genre_1,"Genre2":genre_2,"Genre3":genre_3, "Gross Collection":gross_collection,"Certification":certification,"votes":votes})
Dataset2

,Movie Name,Star1,Star2,Star3,Star4,Genre1,Genre2,Genre3,Gross Collection,Certification,votes
0,Spider-Man: Across the Spider-Verse,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure,Nan,PG,"54,872"
1,The Dark Knight,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama,534.86,PG-13,"2,720,281"
2,The Lord of the Rings: The Return of the King,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama,377.85,PG-13,"1,888,273"
3,Inception,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi,292.58,PG-13,"2,414,702"
4,The Lord of the Rings: The Fellowship of the Ring,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama,315.54,PG-13,"1,916,881"
...,...,...,...,...,...,...,...,...,...,...,...
1746,Radhe,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,Action,Crime,Thriller,Nan,TV-MA,"178,817"
1747,Race 3,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,Action,Crime,Thriller,1.69,Not Rated,"47,593"
1748,Angels Apocalypse,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,Action,Fantasy,Sci-Fi,Nan,Nan,"42,911"
1749,The Cost of Deception,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,Action,Crime,Drama,Nan,Nan,"39,491"


In [12]:
dataset1.to_csv('data1.csv')

In [13]:
Dataset2.to_csv('data2.csv')